## 필수과제 1.4
### 베스트 제품의 이벤트별 차이 -> 월별에 따른 이벤트별 베스트 제품이 달라지는가? 인사이트 정리
* 월별로 event_type 단위로 best item 과 brand 를 산출
    * 월별로 순위가 달라지는지 확인
        * Top10 View items
            * 10월 ~ 12월 간 15개의 unique item 이 도출
            * 15개 중 6개의 제품은 3달간 top10 안에 포함
            * product_id 5809910 는 3달간 높은 순위에 rank
            
        * Top10 Cart items
            * 10월 ~ 12월 간 14개의 unique item 이 도출
            * 14개 중 5개의 제품은 3달간 top10 안에 포함

        * Top10 Purchase items
            * 10월 ~ 12월 간 13개의 unique item 이 도출
            * 13개 중 7개의 제품은 3달간 top10 안에 포함됨 (-> 인기도 기준의 추천이 상대적으로 효과적일수 있음)

        * 10월
            * top10 view 상품 중에 top10 cart/purchase 에 드는 상품은 1개
            * top10 cart 상품 중에 top10 purchase 에 드는 상품은 8개
        * 11월
            * top10 view 상품 중에 top10 cart에 드는 상품은 3개 , top10 purchase 에 드는 상품은 4개
            * top10 cart 상품 중에 top10 purchase 에 드는 상품은 8개
        * 12월
            * top10 view 상품 중에 top10 cart/purchase 에 드는 상품은 2개
            * top10 cart 상품 중에 top10 purchase 에 드는 상품은 7개
    
    * Brand
        * 동일하게 브랜드 기준으로 top10을 view/cart/purcahse event 단위로 산출시 product level 대비 변동이 적은 것을 알 수 있음
        * 월별로 구매 event 기준 top10 안에 든 unique 브랜드 수는 12개 이며, 이중 6개는 3달 동안 1~6위 안에서 변동할 정도로 브랜드 파워가 상대적으로 강한 제품군이 존재함

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display 
sns.set_style('whitegrid')

In [2]:
df1 = pd.read_csv("../dataset/2019-Oct.csv")
df2 = pd.read_csv("../dataset/2019-Nov.csv")
df3 = pd.read_csv("../dataset/2019-Dec.csv")

In [8]:
df1.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,cart,5773203,1487580005134238553,NaN,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885
1,2019-10-01 00:00:03 UTC,cart,5773353,1487580005134238553,NaN,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885
2,2019-10-01 00:00:07 UTC,cart,5881589,2151191071051219817,NaN,lovely,13.48,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9
3,2019-10-01 00:00:07 UTC,cart,5723490,1487580005134238553,NaN,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885
4,2019-10-01 00:00:15 UTC,cart,5881449,1487580013522845895,NaN,lovely,0.56,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9


In [16]:
total_items = pd.concat([
    df1[['product_id','category_id','brand','price']]
    , df2[['product_id','category_id','brand','price']]
    , df3[['product_id','category_id','brand','price']]
    ]).drop_duplicates()
total_items.head()

,product_id,category_id,brand,price
0,5773203,1487580005134238553,runail,2.62
1,5773353,1487580005134238553,runail,2.62
2,5881589,2151191071051219817,lovely,13.48
3,5723490,1487580005134238553,runail,2.62
4,5881449,1487580013522845895,lovely,0.56


In [17]:
print(total_items.shape)
print(total_items['product_id'].nunique())

(101752, 4)
49341


In [20]:
total_items.sort_values(by='product_id')

,product_id,category_id,brand,price
59027,3752,1487580005411062629,cnd,15.71
4826,3762,1487580005411062629,cnd,19.37
2800240,3762,1487580005411062629,cnd,18.40
40458,3763,1487580005411062629,cnd,16.03
2828196,3763,1487580005411062629,cnd,15.54
...,...,...,...,...
3473603,5917175,1487580012096782476,herbolive,9.02
3457607,5917175,1487580012096782476,NaN,0.00
3457622,5917177,1487580011627020412,NaN,0.00
3470836,5917177,1487580011627020412,herbolive,19.81


In [31]:
def monthly_event_topn(df, k=5):
    top_viewed=df[df['event_type']=='view']['product_id'].value_counts().head(k).reset_index()
    top_carted=df[df['event_type']=='cart']['product_id'].value_counts().head(k).reset_index()
    top_purchased=df[df['event_type']=='purchase']['product_id'].value_counts().head(k).reset_index()

    top_viewed = top_viewed.reset_index(names='rank')
    top_carted = top_carted.reset_index(names='rank')
    top_purchased = top_purchased.reset_index(names='rank')

    return top_viewed,top_carted,top_purchased

In [32]:
oct_top_viewed, oct_top_carted, oct_top_purchased = monthly_event_topn(df1, k=10)
nov_top_viewed, nov_top_carted, nov_top_purchased = monthly_event_topn(df2, k=10)
dec_top_viewed, dec_top_carted, dec_top_purchased = monthly_event_topn(df3, k=10)

In [34]:
# Top view items OCT ~ DEC
top_view_items_rank = pd.merge(oct_top_viewed,nov_top_viewed, on='product_id', how='outer')
top_view_items_rank = pd.merge(top_view_items_rank,dec_top_viewed, on='product_id', how='outer')
top_view_items_rank

,rank_x,product_id,count_x,rank_y,count_y,rank,count
0,0.0,5892179,9013.0,NaN,NaN,NaN,NaN
1,1.0,5809910,8473.0,0.0,18814.0,0.0,24419.0
2,2.0,5877454,6597.0,6.0,4961.0,2.0,6621.0
3,3.0,5886282,4989.0,2.0,7337.0,4.0,5488.0
4,4.0,5809912,4936.0,3.0,6380.0,3.0,5944.0
5,5.0,5877456,4816.0,7.0,4755.0,5.0,5022.0
6,6.0,5751383,4715.0,4.0,5245.0,NaN,NaN
7,7.0,5856186,4547.0,9.0,4508.0,9.0,4015.0
8,8.0,5649236,4231.0,NaN,NaN,6.0,4372.0
9,9.0,5809911,4220.0,NaN,NaN,7.0,4307.0


In [37]:
# Top cart items OCT ~ DEC
top_cart_items_rank = pd.merge(oct_top_carted,nov_top_carted, on='product_id', how='outer')
top_cart_items_rank = pd.merge(top_cart_items_rank,dec_top_carted, on='product_id', how='outer')
top_cart_items_rank

,rank_x,product_id,count_x,rank_y,count_y,rank,count
0,0.0,5700037,4998.0,1.0,6683.0,2.0,4978.0
1,1.0,5802432,4693.0,3.0,5932.0,1.0,5081.0
2,2.0,5686925,4356.0,8.0,3070.0,NaN,NaN
3,3.0,5854897,3966.0,2.0,6083.0,3.0,4078.0
4,4.0,5700046,2897.0,5.0,3655.0,9.0,2035.0
5,5.0,5815662,2830.0,4.0,4350.0,4.0,3047.0
6,6.0,5688124,2745.0,NaN,NaN,NaN,NaN
7,7.0,5836522,2470.0,NaN,NaN,6.0,2171.0
8,8.0,5751422,2251.0,7.0,3167.0,NaN,NaN
9,9.0,5751383,2068.0,NaN,NaN,NaN,NaN


In [36]:
# Top purchase items OCT ~ DEC
top_order_items_rank = pd.merge(oct_top_purchased,nov_top_purchased, on='product_id', how='outer')
top_order_items_rank = pd.merge(top_order_items_rank,dec_top_purchased, on='product_id', how='outer')
top_order_items_rank

,rank_x,product_id,count_x,rank_y,count_y,rank,count
0,0.0,5854897,700.0,1.0,1165.0,1.0,786.0
1,1.0,5700037,695.0,4.0,873.0,3.0,621.0
2,2.0,5751422,644.0,3.0,934.0,7.0,548.0
3,3.0,5802432,640.0,8.0,805.0,2.0,714.0
4,4.0,5792800,592.0,NaN,NaN,NaN,NaN
5,5.0,5686925,583.0,NaN,NaN,NaN,NaN
6,6.0,5751383,566.0,9.0,802.0,9.0,435.0
7,7.0,5304,524.0,6.0,865.0,6.0,549.0
8,8.0,5815662,498.0,5.0,866.0,8.0,521.0
9,9.0,5688124,462.0,NaN,NaN,NaN,NaN


In [40]:
display(pd.merge(oct_top_viewed, oct_top_carted,on='product_id',how='inner'))
display(pd.merge(oct_top_carted, oct_top_purchased,on='product_id',how='inner'))
display(pd.merge(oct_top_viewed, oct_top_purchased,on='product_id',how='inner'))


,rank_x,product_id,count_x,rank_y,count_y
0,6,5751383,4715,9,2068


,rank_x,product_id,count_x,rank_y,count_y
0,0,5700037,4998,1,695
1,1,5802432,4693,3,640
2,2,5686925,4356,5,583
3,3,5854897,3966,0,700
4,5,5815662,2830,8,498
5,6,5688124,2745,9,462
6,8,5751422,2251,2,644
7,9,5751383,2068,6,566


,rank_x,product_id,count_x,rank_y,count_y
0,6,5751383,4715,6,566


In [41]:
display(pd.merge(nov_top_viewed, nov_top_carted,on='product_id',how='inner'))
display(pd.merge(nov_top_carted, nov_top_purchased,on='product_id',how='inner'))
display(pd.merge(nov_top_viewed, nov_top_purchased,on='product_id',how='inner'))


,rank_x,product_id,count_x,rank_y,count_y
0,0,5809910,18814,0,7467
1,3,5809912,6380,6,3327
2,5,5751422,5206,7,3167


,rank_x,product_id,count_x,rank_y,count_y
0,0,5809910,7467,0,2138
1,1,5700037,6683,4,873
2,2,5854897,6083,1,1165
3,3,5802432,5932,8,805
4,4,5815662,4350,5,866
5,6,5809912,3327,7,862
6,7,5751422,3167,3,934
7,9,5304,2929,6,865


,rank_x,product_id,count_x,rank_y,count_y
0,0,5809910,18814,0,2138
1,3,5809912,6380,7,862
2,4,5751383,5245,9,802
3,5,5751422,5206,3,934


In [42]:
display(pd.merge(dec_top_viewed, dec_top_carted,on='product_id',how='inner'))
display(pd.merge(dec_top_carted, dec_top_purchased,on='product_id',how='inner'))
display(pd.merge(dec_top_viewed, dec_top_purchased,on='product_id',how='inner'))


,rank_x,product_id,count_x,rank_y,count_y
0,0,5809910,24419,0,6513
1,3,5809912,5944,5,2634


,rank_x,product_id,count_x,rank_y,count_y
0,0,5809910,6513,0,1659
1,1,5802432,5081,2,714
2,2,5700037,4978,3,621
3,3,5854897,4078,1,786
4,4,5815662,3047,8,521
5,5,5809912,2634,4,620
6,8,5304,2036,6,549


,rank_x,product_id,count_x,rank_y,count_y
0,0,5809910,24419,0,1659
1,3,5809912,5944,4,620


In [51]:
def monthly_event_topn_brand(df, k=5):
    df = df[df['brand'].isnull()==False]
    top_viewed=df[df['event_type']=='view']['brand'].value_counts().head(k).reset_index()
    top_carted=df[df['event_type']=='cart']['brand'].value_counts().head(k).reset_index()
    top_purchased=df[df['event_type']=='purchase']['brand'].value_counts().head(k).reset_index()

    top_viewed = top_viewed.reset_index(names='rank')
    top_carted = top_carted.reset_index(names='rank')
    top_purchased = top_purchased.reset_index(names='rank')

    return top_viewed,top_carted,top_purchased

oct_top_brn_viewed, oct_top_brn_carted, oct_top_brn_purchased = monthly_event_topn_brand(df1, k=10)
nov_top_brn_viewed, nov_top_brn_carted, nov_top_brn_purchased = monthly_event_topn_brand(df2, k=10)
dec_top_brn_viewed, dec_top_brn_carted, dec_top_brn_purchased = monthly_event_topn_brand(df3, k=10)

In [49]:
# Top view brands OCT ~ DEC
top_view_brand_rank = pd.merge(oct_top_brn_viewed,nov_top_brn_viewed, on='brand', how='outer')
top_view_brand_rank = pd.merge(top_view_brand_rank,dec_top_brn_viewed, on='brand', how='outer')
top_view_brand_rank

,rank_x,brand,count_x,rank_y,count_y,rank,count
0,0.0,runail,119609.0,0.0,124147.0,0.0,108097.0
1,1.0,irisk,85321.0,2.0,78036.0,2.0,66529.0
2,2.0,masura,79508.0,3.0,69273.0,3.0,50363.0
3,3.0,grattol,67871.0,1.0,96159.0,1.0,83257.0
4,4.0,bpw.style,43325.0,4.0,47424.0,7.0,34371.0
5,5.0,estel,36989.0,6.0,34696.0,5.0,44300.0
6,6.0,ingarden,35091.0,5.0,40777.0,8.0,28253.0
7,7.0,kapous,32916.0,8.0,32250.0,4.0,45225.0
8,8.0,jessnail,29100.0,7.0,34008.0,6.0,34913.0
9,9.0,concept,25665.0,NaN,NaN,9.0,23363.0


In [50]:
# Top cart brands OCT ~ DEC
top_cart_brand_rank = pd.merge(oct_top_brn_carted,nov_top_brn_carted, on='brand', how='outer')
top_cart_brand_rank = pd.merge(top_cart_brand_rank,dec_top_brn_carted, on='brand', how='outer')
top_cart_brand_rank

,rank_x,brand,count_x,rank_y,count_y,rank,count
0,0.0,runail,98737.0,0.0,108769.0,0.0,80411.0
1,1.0,irisk,78293.0,1.0,64621.0,1.0,45841.0
2,2.0,masura,58657.0,2.0,53991.0,3.0,33788.0
3,3.0,bpw.style,37785.0,4.0,41635.0,4.0,21995.0
4,4.0,grattol,33494.0,3.0,53553.0,2.0,37841.0
5,5.0,ingarden,27948.0,5.0,29620.0,6.0,17207.0
6,6.0,pole,19789.0,NaN,NaN,8.0,13452.0
7,7.0,estel,16207.0,NaN,NaN,5.0,17946.0
8,8.0,kapous,14576.0,8.0,15723.0,7.0,13751.0
9,9.0,italwax,13925.0,9.0,14972.0,NaN,NaN


In [52]:
# Top cart brands OCT ~ DEC
top_order_brand_rank = pd.merge(oct_top_brn_purchased,nov_top_brn_purchased, on='brand', how='outer')
top_order_brand_rank = pd.merge(top_order_brand_rank,dec_top_brn_purchased, on='brand', how='outer')
top_order_brand_rank

,rank_x,brand,count_x,rank_y,count_y,rank,count
0,0.0,runail,21936.0,0.0,25999.0,0.0,18199.0
1,1.0,irisk,15645.0,1.0,16368.0,1.0,10583.0
2,2.0,masura,11356.0,4.0,11481.0,4.0,6985.0
3,3.0,bpw.style,9506.0,3.0,12986.0,3.0,7014.0
4,4.0,grattol,6438.0,2.0,13613.0,2.0,8171.0
5,5.0,ingarden,5181.0,5.0,7797.0,5.0,4157.0
6,6.0,italwax,3546.0,8.0,4306.0,NaN,NaN
7,7.0,pole,3360.0,NaN,NaN,8.0,3080.0
8,8.0,uno,3218.0,7.0,4717.0,9.0,2780.0
9,9.0,estel,3211.0,NaN,NaN,6.0,4116.0
